In [1]:
import cv2
from skimage import io
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import math
import sys
import pickle
import random
from haar_like_features import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
class CascadeClassifier():
    def __init__(self, layers):
        """
        layers: list of int 
            count of how many featuer in classifiers
            eg: [10, 20, 30]
        clfs: list of strong classifiers
        """
        self.layers = layers
        self.clfs = []